# Import libraries

In [1]:
# Data Manipulation
import numpy as np
import pandas as pd

# Modelling
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-09 17:31:23.845834: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-09 17:31:24.143294: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-09 17:31:24.251844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736443884.351770   13609 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736443884.379817   13609 cuda_blas.cc:14

# Import data

In [2]:
df = pd.read_csv("data/export/net_amount_discount_vs_method_monthly.csv", header=0)
df.head()

,Purchase Date,Purchase Method,Amount Type,Amount
0,2019-09-01,Cash on Delivery,Net Amount,33779.539
1,2019-09-01,Cash on Delivery,Discount Amount INR,2387.060
2,2019-09-01,Credit Card,Net Amount,332777.303
3,2019-09-01,Credit Card,Discount Amount INR,13813.480
4,2019-09-01,Debit Card,Net Amount,246856.295


# Data Manipulation

 - Extend the data to create a complete timeline of the data
 - Interpolate missing values

In [3]:
# Convert Purchase Date to datetime
df["Purchase Date"] = pd.to_datetime(df["Purchase Date"])

# Get unique amount types and purchase methods
amount_types = df["Amount Type"].unique()
purchase_methods = df["Purchase Method"].unique()

# Create an empty dataframe to store the results
result_df = pd.DataFrame()

# Process each amount type and purchase method
for amount_type in amount_types:
    for purchase_method in purchase_methods:
        # Filter data for current amount type and purchase method
        type_method_data = df[(df["Amount Type"] == amount_type) & 
                              (df["Purchase Method"] == purchase_method)].copy()
        if type_method_data.empty:
            continue
        # Get min and max dates for this amount type and purchase method
        min_date = type_method_data["Purchase Date"].min()
        max_date = type_method_data["Purchase Date"].max()
        # Create date range specific to this amount type and purchase method
        date_range = pd.date_range(start=min_date, end=max_date, freq="MS")
        # Create complete timeline for this amount type and purchase method
        complete_df = pd.DataFrame({"Purchase Date": date_range})
        complete_df = complete_df.merge(type_method_data, on="Purchase Date", how="left")
        # Fill missing Amount Type and Purchase Method
        complete_df["Amount Type"] = amount_type
        complete_df["Purchase Method"] = purchase_method
        # Fill missing values with NaN
        complete_df["Amount"] = complete_df["Amount"].fillna(np.nan)
        # Interpolate missing values
        complete_df["Amount"] = complete_df["Amount"].interpolate(method="linear", limit_direction="both")
        # Append to the result dataframe
        result_df = pd.concat([result_df, complete_df])

# Sort the final dataframe
result_df = result_df.sort_values(["Purchase Date", "Amount Type", "Purchase Method"]).reset_index(drop=True)

# Forecasting

## Prophet

Prophet model used to fit on monthly data considering monthly seasonality.

In [5]:
# Create empty dataframe to store forecasts
forecast_df = pd.DataFrame()

# Process each amount type and purchase method for forecasting
for amount_type in amount_types:
    for purchase_method in purchase_methods:
        # Prepare data for Prophet
        prophet_df = result_df[(result_df["Amount Type"] == amount_type) & 
                               (result_df["Purchase Method"] == purchase_method)].copy()
        if prophet_df.empty:
            continue
        prophet_df = prophet_df[["Purchase Date", "Amount"]]
        prophet_df.columns = ["ds", "y"]
        # Initialize and fit Prophet model
        model = Prophet(yearly_seasonality=True)
        model.add_seasonality(name="monthly", period=30.5, fourier_order=5)
        model.fit(prophet_df)
        # Make future dataframe for forecasting
        future = model.make_future_dataframe(periods=12, freq="MS")
        # Forecast
        forecast = model.predict(future)
        # Add amount type and purchase method columns
        forecast["Amount Type"] = amount_type
        forecast["Purchase Method"] = purchase_method
        # Store forecast
        forecast_df = pd.concat([forecast_df, forecast])

17:38:32 - cmdstanpy - INFO - Chain [1] start processing
17:38:33 - cmdstanpy - INFO - Chain [1] done processing
17:38:33 - cmdstanpy - INFO - Chain [1] start processing
17:38:33 - cmdstanpy - INFO - Chain [1] done processing
17:38:33 - cmdstanpy - INFO - Chain [1] start processing
17:38:33 - cmdstanpy - INFO - Chain [1] done processing
17:38:33 - cmdstanpy - INFO - Chain [1] start processing
17:38:34 - cmdstanpy - INFO - Chain [1] done processing
17:38:34 - cmdstanpy - INFO - Chain [1] start processing
17:38:34 - cmdstanpy - INFO - Chain [1] done processing
17:38:34 - cmdstanpy - INFO - Chain [1] start processing
17:38:34 - cmdstanpy - INFO - Chain [1] done processing
17:38:34 - cmdstanpy - INFO - Chain [1] start processing
17:38:34 - cmdstanpy - INFO - Chain [1] done processing
17:38:34 - cmdstanpy - INFO - Chain [1] start processing
17:38:35 - cmdstanpy - INFO - Chain [1] done processing
17:38:35 - cmdstanpy - INFO - Chain [1] start processing
17:38:35 - cmdstanpy - INFO - Chain [1]

### Actual vs Forecast Plot

In [8]:
# Create subplot for Discount Amount INR
fig_discount = make_subplots(rows=len(purchase_methods), cols=1, subplot_titles=purchase_methods)

# Plot each purchase method for Discount Amount INR
for i, purchase_method in enumerate(purchase_methods, 1):
    # Get data for Discount Amount INR and current purchase method
    actual_data_discount = result_df[(result_df["Amount Type"] == "Discount Amount INR") & (result_df["Purchase Method"] == purchase_method)]
    forecast_data_discount = forecast_df[(forecast_df["Amount Type"] == "Discount Amount INR") & (forecast_df["Purchase Method"] == purchase_method)]
    # Add traces for Discount Amount INR
    fig_discount.add_trace(
        go.Scatter(
            x=actual_data_discount["Purchase Date"],
            y=actual_data_discount["Amount"],
            name=f"Actual - Discount Amount INR - {purchase_method}",
            mode="lines",
        ),
        row=i,
        col=1,
    )
    fig_discount.add_trace(
        go.Scatter(
            x=forecast_data_discount["ds"],
            y=forecast_data_discount["yhat"],
            name=f"Forecast - Discount Amount INR - {purchase_method}",
            mode="lines",
        ),
        row=i,
        col=1,
    )
    fig_discount.add_trace(
        go.Scatter(
            x=forecast_data_discount["ds"],
            y=forecast_data_discount["yhat_upper"],
            fill=None,
            mode="lines",
            line_color="rgba(0,100,80,0.2)",
            showlegend=False,
        ),
        row=i,
        col=1,
    )
    fig_discount.add_trace(
        go.Scatter(
            x=forecast_data_discount["ds"],
            y=forecast_data_discount["yhat_lower"],
            fill="tonexty",
            mode="lines",
            line_color="rgba(0,100,80,0.2)",
            name="Confidence Interval",
        ),
        row=i,
        col=1,
    )

# Update layout for Discount Amount INR plot
fig_discount.update_layout(height=800 * len(purchase_methods), title_text="Forecast vs Actual for Discount Amount INR")
fig_discount.show()

# Create subplot for Net Amount
fig_net_amount = make_subplots(rows=len(purchase_methods), cols=1, subplot_titles=purchase_methods)

# Plot each purchase method for Net Amount
for i, purchase_method in enumerate(purchase_methods, 1):
    # Get data for Net Amount and current purchase method
    actual_data_net = result_df[(result_df["Amount Type"] == "Net Amount") & (result_df["Purchase Method"] == purchase_method)]
    forecast_data_net = forecast_df[(forecast_df["Amount Type"] == "Net Amount") & (forecast_df["Purchase Method"] == purchase_method)]
    # Add traces for Net Amount
    fig_net_amount.add_trace(
        go.Scatter(
            x=actual_data_net["Purchase Date"],
            y=actual_data_net["Amount"],
            name=f"Actual - Net Amount - {purchase_method}",
            mode="lines",
        ),
        row=i,
        col=1,
    )
    fig_net_amount.add_trace(
        go.Scatter(
            x=forecast_data_net["ds"],
            y=forecast_data_net["yhat"],
            name=f"Forecast - Net Amount - {purchase_method}",
            mode="lines",
        ),
        row=i,
        col=1,
    )
    fig_net_amount.add_trace(
        go.Scatter(
            x=forecast_data_net["ds"],
            y=forecast_data_net["yhat_upper"],
            fill=None,
            mode="lines",
            line_color="rgba(0,100,80,0.2)",
            showlegend=False,
        ),
        row=i,
        col=1,
    )
    fig_net_amount.add_trace(
        go.Scatter(
            x=forecast_data_net["ds"],
            y=forecast_data_net["yhat_lower"],
            fill="tonexty",
            mode="lines",
            line_color="rgba(0,100,80,0.2)",
            name="Confidence Interval",
        ),
        row=i,
        col=1,
    )

# Update layout for Net Amount plot
fig_net_amount.update_layout(height=800 * len(purchase_methods), title_text="Forecast vs Actual for Net Amount")
fig_net_amount.show()

## Neural Network

LSTM been used for forecasting for next 12 months

In [9]:
# Prepare data for neural network
scaler = MinMaxScaler(feature_range=(0, 1))
result_df['Amount_scaled'] = scaler.fit_transform(result_df[['Amount']])

# Function to create dataset for LSTM
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

# Forecasting using LSTM
forecast_nn_df = pd.DataFrame()
predicted_actual_df = pd.DataFrame()

for amount_type in amount_types:
    for purchase_method in purchase_methods:
        # Filter data for current amount type and purchase method
        type_method_data = result_df[(result_df["Amount Type"] == amount_type) & 
                                     (result_df["Purchase Method"] == purchase_method)].copy()
        if type_method_data.empty:
            continue
        type_method_data = type_method_data[['Purchase Date', 'Amount_scaled']]
        # Create dataset
        time_step = 12
        X, Y = create_dataset(type_method_data['Amount_scaled'].values.reshape(-1, 1), time_step)
        # Reshape input to be [samples, time steps, features] which is required for LSTM
        X = X.reshape(X.shape[0], X.shape[1], 1)
        # Create LSTM model
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))
        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')
        # Train the model
        model.fit(X, Y, batch_size=1, epochs=10, verbose=1)
        # Predict for actual period
        predicted_actual = model.predict(X)
        predicted_actual = scaler.inverse_transform(predicted_actual)
        # Create actual prediction dataframe
        actual_dates = type_method_data['Purchase Date'].iloc[time_step + 1:]
        actual_df = pd.DataFrame({
            'Purchase Date': actual_dates,
            'Amount': predicted_actual.flatten(),
            'Amount Type': amount_type,
            'Purchase Method': purchase_method
        })
        predicted_actual_df = pd.concat([predicted_actual_df, actual_df])
        # Forecast for next 12 months
        temp_input = type_method_data['Amount_scaled'].values[-time_step:].tolist()
        lst_output = []
        i = 0
        while i < 12:
            x_input = np.array(temp_input[-time_step:]).reshape((1, time_step, 1))
            yhat = model.predict(x_input, verbose=0)
            temp_input.append(yhat[0][0])
            lst_output.append(yhat[0][0])
            i += 1
        # Create forecast dataframe
        forecast_dates = pd.date_range(start=type_method_data['Purchase Date'].max(), periods=13, freq='MS')[1:]
        forecast_amounts = scaler.inverse_transform(np.array(lst_output).reshape(-1, 1))
        forecast_nn = pd.DataFrame({
            'Purchase Date': forecast_dates,
            'Amount': forecast_amounts.flatten(),
            'Amount Type': amount_type,
            'Purchase Method': purchase_method
        })
        forecast_nn_df = pd.concat([forecast_nn_df, forecast_nn])


Epoch 1/10


2025-01-09 17:48:52.292233: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0016
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.0047e-04
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.2902e-04
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.0171e-04
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.0480e-04
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.6886e-04
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.6489e-04
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.8552e-04
Epoch 9/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.2602e-04
Epoch 10/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.3203e-04
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
Epoch 1/10


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1526
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0180
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0156
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0222  
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0238   
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0277
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0198
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0149
Epoch 9/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0205
Epoch 10/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0222
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Epoch 1/10


...

48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 3.6143e-05
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.2648e-06
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2202e-06
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.1237e-06
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.5484e-06
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.2594e-06
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.3171e-06
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.4458e-06
Epoch 9/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.4504e-06
Epoch 10/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.8213e-06
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Epoch 1/10


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 4.7039e-05
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.5241e-06
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.2494e-06
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.9820e-06
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.6713e-06
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.0826e-06
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2682e-06
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.1068e-06
Epoch 9/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6504e-06
Epoch 10/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5219e-06
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
Epoch 1/10


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 8.1222e-05
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.0733e-06
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.0736e-06
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.5806e-06
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.9434e-06
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7047e-06
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.8174e-06
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.8568e-06
Epoch 9/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.7069e-06
Epoch 10/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.0576e-06
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step


### Actual vs Forecast Plot

In [12]:
# Plot actual, predicted, and forecasted data for Discount Amount INR
fig_discount_nn = make_subplots(rows=len(purchase_methods), cols=1, subplot_titles=purchase_methods)

for i, purchase_method in enumerate(purchase_methods, 1):
    actual_data_discount = result_df[(result_df["Amount Type"] == "Discount Amount INR") & (result_df["Purchase Method"] == purchase_method)]
    predicted_actual_data_discount = predicted_actual_df[(predicted_actual_df["Amount Type"] == "Discount Amount INR") & (predicted_actual_df["Purchase Method"] == purchase_method)]
    forecast_data_discount = forecast_nn_df[(forecast_nn_df["Amount Type"] == "Discount Amount INR") & (forecast_nn_df["Purchase Method"] == purchase_method)]
    fig_discount_nn.add_trace(
        go.Scatter(
            x=actual_data_discount["Purchase Date"],
            y=actual_data_discount["Amount"],
            name=f"Actual - Discount Amount INR - {purchase_method}",
            mode="lines",
        ),
        row=i,
        col=1,
    )
    fig_discount_nn.add_trace(
        go.Scatter(
            x=predicted_actual_data_discount["Purchase Date"],
            y=predicted_actual_data_discount["Amount"],
            name=f"Predicted Actual - Discount Amount INR - {purchase_method}",
            mode="lines",
        ),
        row=i,
        col=1,
    )
    fig_discount_nn.add_trace(
        go.Scatter(
            x=forecast_data_discount["Purchase Date"],
            y=forecast_data_discount["Amount"],
            name=f"Forecast - Discount Amount INR - {purchase_method}",
            mode="lines",
        ),
        row=i,
        col=1,
    )

fig_discount_nn.update_layout(height=800 * len(purchase_methods), title_text="Actual, Predicted Actual, and Forecast for Discount Amount INR using Neural Network")
fig_discount_nn.show()

# Plot actual, predicted, and forecasted data for Net Amount
fig_net_nn = make_subplots(rows=len(purchase_methods), cols=1, subplot_titles=purchase_methods)

for i, purchase_method in enumerate(purchase_methods, 1):
    actual_data_net = result_df[(result_df["Amount Type"] == "Net Amount") & (result_df["Purchase Method"] == purchase_method)]
    predicted_actual_data_net = predicted_actual_df[(predicted_actual_df["Amount Type"] == "Net Amount") & (predicted_actual_df["Purchase Method"] == purchase_method)]
    forecast_data_net = forecast_nn_df[(forecast_nn_df["Amount Type"] == "Net Amount") & (forecast_nn_df["Purchase Method"] == purchase_method)]
    fig_net_nn.add_trace(
        go.Scatter(
            x=actual_data_net["Purchase Date"],
            y=actual_data_net["Amount"],
            name=f"Actual - Net Amount - {purchase_method}",
            mode="lines",
        ),
        row=i,
        col=1,
    )
    fig_net_nn.add_trace(
        go.Scatter(
            x=predicted_actual_data_net["Purchase Date"],
            y=predicted_actual_data_net["Amount"],
            name=f"Predicted Actual - Net Amount - {purchase_method}",
            mode="lines",
        ),
        row=i,
        col=1,
    )
    fig_net_nn.add_trace(
        go.Scatter(
            x=forecast_data_net["Purchase Date"],
            y=forecast_data_net["Amount"],
            name=f"Forecast - Net Amount - {purchase_method}",
            mode="lines",
        ),
        row=i,
        col=1,
    )

fig_net_nn.update_layout(height=800 * len(purchase_methods), title_text="Actual, Predicted Actual, and Forecast for Net Amount using Neural Network")
fig_net_nn.show()